In [2]:
import sys
import json
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.auto import tqdm
from tqdm.keras import TqdmCallback
sys.path.append("../../../")
from src.Evaluation import plot_model_history
from src.ModelBuilder import get_MLP, get_MCDCNN, get_Time_CNN, get_FCN, get_Encoder
from src.LoadData import get_all_datasets_test_train_np_arrays
from src.Helpers import append_to_csv

2023-01-21 17:11:02.684667: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-21 17:11:07.414188: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-21 17:11:24.845243: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.8/lib64
2023-01-21 17:11:24.855179: W tensorflow/compil

In [3]:
%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/bwhpc/common/devel/cuda/11.8

env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/bwhpc/common/devel/cuda/11.8


In [4]:
path_to_datasets = "../../../datasets"

In [5]:
datasets_test_train_data = get_all_datasets_test_train_np_arrays(path_to_datasets)

In [6]:
models_getter = [get_Encoder, ]
models_names = ["Encoder", ]

In [6]:
len(datasets_test_train_data)

33

In [7]:
# tf.debugging.set_log_device_placement(True)
devices = tf.config.list_logical_devices('GPU') # Use this to run training just on GPUs
#devices = tf.config.list_logical_devices()
strategy = tf.distribute.MirroredStrategy(devices)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2023-01-21 16:28:18.686148: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.8/lib64
2023-01-21 16:28:18.686190: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-21 16:28:18.686214: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (uc2n369.localdomain): /proc/driver/nvidia/version does not exist
2023-01-21 16:28:18.723468: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate co

In [8]:
devices

[]

In [9]:
path_persist_results = "./training_res.csv"
if not (csv_path := Path(path_persist_results)).exists():
    csv_path.touch()
if len(csv_path.read_text()) == 0:
    # Only write if the file is empty
    columns = ["dataset_name", "model_name", "test_loss", "test_acc", "confusion_matrix", "history"]
    append_to_csv(path_persist_results, columns)

In [10]:
epochs = 30
batch_size = 10
validation_split = 0.2

In [ ]:
with strategy.scope():
    for ds_name, ds_data in tqdm(datasets_test_train_data.items(), unit='dataset'):
        print("Dataset name: ", ds_name)
        x_test, y_test = ds_data["test_data"]
        x_train, y_train = ds_data["train_data"]

        input_size = x_train.shape[1]
        output_size = len(np.unique(y_train))

        for get_model, model_name in tqdm(list(zip(models_getter, models_names)), unit='model', desc=f'Train on "{ds_name}"'):
            print("Model name: ", model_name)
            model = get_model(input_size, output_size)
            model.compile(optimizer='SGD',
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])
            history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split, callbacks=[TqdmCallback(verbose=0, desc=model_name)], verbose=0)
            test_loss, test_acc = model.evaluate(x_test, y_test)
            # TODO: add confusion matrix

            row = [ds_name,
                  model_name,
                  test_loss,
                  test_acc,
                  None, # TODO: persist confusion matrix to further analysis
                  json.dumps(history.history)]
            append_to_csv(path_persist_results, row)

  0%|          | 0/33 [00:00<?, ?dataset/s]

Dataset name:  cbf


Train on "cbf":   0%|          | 0/1 [00:00<?, ?model/s]

Model name:  Encoder


Encoder: 0epoch [00:00, ?epoch/s]

...

2/2 [==============================] - 1s 302ms/step - loss: 0.8588 - accuracy: 0.6500
Dataset name:  abnormal_heartbeat


Train on "abnormal_heartbeat":   0%|          | 0/1 [00:00<?, ?model/s]

Model name:  Encoder


Encoder: 0epoch [00:00, ?epoch/s]

2023-01-21 16:34:49.353667: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2023-01-21 16:38:55.973658: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


# Example load training results and display evaluation results

In [ ]:
df_training_res = pd.read_csv(path_persist_results)

In [ ]:
df_training_res

In [ ]:
history = df_training_res['history'][0]

In [15]:
json.loads(history)

{'loss': [0.5163329243659973,
  0.0538581945002079,
  0.00482696108520031,
  0.001751854782924056,
  0.0016706426395103335,
  0.0007307759369723499,
  0.0006830706843174994,
  0.0004944929969497025,
  0.0004373743140604347,
  0.0009648362174630165,
  0.0005137254483997822,
  0.00031668427982367575,
  0.00028367075719870627,
  0.00027524339384399354,
  0.00031098644831217825,
  0.00021740842203143984,
  0.00024244197993539274,
  0.00027552287792786956,
  0.00020925352873746306,
  0.0001911270956043154,
  0.0002227224176749587,
  0.00016026344383135438,
  0.00018987749353982508,
  0.00026405006065033376,
  0.00015174208965618163,
  9.252057498088107e-05,
  0.0001497379707871005,
  0.0001423564535798505,
  0.0001247290929313749,
  0.0001800397876650095],
 'accuracy': [0.7361963391304016,
  0.9815950989723206,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,


In [ ]:
plot_model_history(json.loads(history), epochs=epochs)